In [1]:
import sys
sys.path.insert(0, '../' )

In [2]:
from libgf264fft import clib_wrapper as cgf

# defining gf
from utils import gf2192 as gf

# the commit scheme
from commit import merkeltreecommit as mt

# hash for challenges
from utils import hash as H

from fri_ldt import m_fri_ldt as fri

def _dummy( *params ) : return None


In [3]:
from utils import randombytes as rd

#rd.randombytes(16)


from aesR1CS import aes128

#pt = _rand_bytes(16)
#sk = _rand_bytes(16)
pt = bytes( [i*16+i for i in range(16)] )
sk = bytes( [i for i in range(16)] )

ct, rd_states , rk , rk_tmp = aes128.aes128_encrypt( pt , sk )

print( sk )
print( pt )
print( rd_states , rk , rk_tmp  )
print( "ct0", ct )



b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
b'\x00\x11"3DUfw\x88\x99\xaa\xbb\xcc\xdd\xee\xff'
b'_rd\x15W\xf5\xbc\x92\xf7\xbe;)\x1d\xb9\xf9\x1a\xff\x87\x96\x841\xd8jQdQQ\xfaw:\xd0\tL\x9c\x1ef\xf7q\xf0v,?\x86\x8eSM\xf2Vc\x85\xb7\x9f\xfcS\x8d\xf9\x97\xbeG\x8euG\xd6\x91\xf4\xbc\xd4T2\xe5T\xd0u\xf1\xd6\xc5\x1d\xd0;<\x98\x16\xeet\x00\xf8\x7fUk,\x04\x9c\x8eZ\xd06\xc5~\x1c\x15\x9a\x9b\xd2\x86\xf0_K\xe0\x98\xc649\xba\xa0=\xe7\xa1\xf9\xb5n\xd5Q,\xba_AM#\xe9\xf7N\xec\x020 \xf6\x1b\xf2\xcc\xf25<!\xc7' b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\xd6\xaat\xfd\xd2\xafr\xfa\xda\xa6x\xf1\xd6\xabv\xfe\xb6\x92\xcf\x0bd=\xbd\xf1\xbe\x9b\xc5\x00h0\xb3\xfe\xb6\xfftN\xd2\xc2\xc9\xbflY\x0c\xbf\x04i\xbfAG\xf7\xf7\xbc\x955>\x03\xf9l2\xbc\xfd\x05\x8d\xfd<\xaa\xa3\xe8\xa9\x9f\x9d\xebP\xf3\xafW\xad\xf6"\xaa^9\x0f}\xf7\xa6\x92\x96\xa7U=\xc1\n\xa3\x1fk\x14\xf9p\x1a\xe3_\xe2\x8cD\n\xdfMN\xa9\xc0&GC\x875\xa4\x1ce\xb9\xe0\x16\xba\xf4\xae\xbfz\xd2T\x992\xd1\xf0\x85Wh\x10\x93\xed\x9c\xb

In [4]:
from aesR1CS import aes128R1CS_z as R1CSz

import importlib
importlib.reload( R1CSz )

z = R1CSz.get_vec_z(pt,sk,64)

print( R1CSz.aes128R1CS_num_constrains , len(z) )


3656 3320


In [5]:
from aesR1CS import r1cs

MATs = r1cs.get_r1cs( R1CSz.aes128R1CS_num_constrains , len(z) , 64, 128)  # AES128

mat_a = MATs['A']
mat_b = MATs['B']
mat_c = MATs['C']


In [6]:
def mat_x_vec( mat , vec , n_row=-1) :
    if 0 > n_row : n_row = len(mat)
    r = [0]*n_row
    for i in range(n_row):
        ri = 0
        for j in range(len(vec)):
            ri ^= cgf.gf264_mul( mat[i][j] , vec[j] )
        r[i] = ri
    return r


In [7]:
import time

st = time.time()
Az = mat_x_vec( mat_a , z )
Bz = mat_x_vec( mat_b , z )
Cz = mat_x_vec( mat_c , z )
ed = time.time()
print( "time:", format(ed-st) , "secs" )


time: 25.3848237991333 secs


In [9]:
import time

def generate_proof( R1CS , h_state , RS_rho = 8 , verbose = 1 ) :
    if 1 == verbose : dump = print
    else : dump = _dummy

    ## process R1CS
    mat_A , mat_B , mat_C , vec_z , witness_idx = R1CS
    n = len(vec_z)
    m = len(mat_A)
    pad_len = 4096
    dump( f"m(#rows): {m} x n: {n}, witness_idx: {witness_idx}, pad_len: {pad_len}" )
    # use global variable
    #dump( "calculate Az, Bz, Cz" )
    #st = time.time()
    #Az = mat_x_vec( mat_A , vec_z )
    #Bz = mat_x_vec( mat_B , vec_z )
    #Cz = mat_x_vec( mat_C , vec_z )
    #ed = time.time()
    #dump( "time:", format(ed-st) , "secs" )
    # XXX: check if Az*Bz == Cz here

    dump( "padding and calcuate f_1v, f_z, f_Az, f_Bz, f_Cz" )
    st = time.time()
    p_vec_z = vec_z + [0]*(pad_len-len(vec_z))     # should pad random gf2192 for zk
    f_z  = cgf.ibtfy( p_vec_z , 0 )
    p_vec_z.extend( cgf.btfy( f_z , pad_len ) )
    f_1v = cgf.ibtfy( vec_z[:witness_idx] , 0 )

    v_Az    = Az + [0]*(pad_len-len(Az))           # should pad random gf2192 for zk
    v_Bz    = Bz + [0]*(pad_len-len(Bz))           # should pad random gf2192 for zk
    v_Cz    = Cz + [0]*(pad_len-len(Cz))           # should pad random gf2192 for zk
    f_Az = cgf.ibtfy( v_Az , 0 )
    f_Bz = cgf.ibtfy( v_Bz , 0 )
    f_Cz = cgf.ibtfy( v_Cz , 0 )
    v_Az.extend( cgf.btfy( f_Az , pad_len ) )
    v_Bz.extend( cgf.btfy( f_Bz , pad_len ) )
    v_Cz.extend( cgf.btfy( f_Bz , pad_len ) )
    ed = time.time()
    dump( "time:", format(ed-st) , "secs" )

    dump( f"f_1v: [{len(f_1v)}]", f_1v )
    dump( f"f_z : [{len(f_z)}] ...[{witness_idx-4}:{witness_idx+4}]:", f_z[witness_idx-4:witness_idx+4] , "..." )
    dump( f"f_w : (f_z - f_1v) /Z_{witness_idx}  <-- XXX: needed op." )

    ## calculate RS code and commit them
    # commit f_w, f_Az, f_Bz , f_Cz, r , r_LDT
    size_L = 1<<19
    offset = 1<<63

    commits = []
    mktrees = []

    ## lincheck
    dump( "lin-check starts. calculate p_alpha, v_alpha" )
    st = time.time()
    v_alpha , v_p2A , v_p2B , v_p2C = [0]*pad_len , [0]*pad_len , [0]*pad_len , [0]*pad_len
    v_alpha[0] = 1
    v_alpha[1] = gf.from_bytes( H.gen( h_state , bytes([1,1]) )[:gf.GF_BSIZE] )
    for i in range(2,m): v_alpha[i] = gf.mul( v_alpha[i-1], v_alpha[1] )
    p_alpha = gf.ifft( v_alpha , 1 , 0 )
    v_alpha.extend( gf.fft(p_alpha,1,pad_len) )
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )

    dump( "lin-check step 1. calculate p2A, p2B, p2C and evaluate their values" )
    st = time.time()
    for j in range(n):
        aj, bj, cj = mat_A[0][j] , mat_B[0][j] , mat_C[0][j]
        for i in range(1,m):
            aj ^= gf.mul_gf264( v_alpha[i] , mat_A[i][j] )
            bj ^= gf.mul_gf264( v_alpha[i] , mat_B[i][j] )
            cj ^= gf.mul_gf264( v_alpha[i] , mat_C[i][j] )
        v_p2A[j] , v_p2B[j] , v_p2C[j] = aj , bj , cj
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )
    st = time.time()
    p2A , p2B , p2C = gf.ifft( v_p2A , 1 , 0 ) , gf.ifft( v_p2B , 1 , 0 ) , gf.ifft( v_p2C , 1 , 0 )
    v_p2A.extend( gf.fft(p2A,1,pad_len) )
    v_p2B.extend( gf.fft(p2B,1,pad_len) )
    v_p2C.extend( gf.fft(p2C,1,pad_len) )
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )

    dump( f"lin-check step 2. poly muls and /Z_{pad_len}" )
    st = time.time()
    v_sA = [ gf.mul(v_Az[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2A[i]) for i in range(2*pad_len) ]
    v_sB = [ gf.mul(v_Bz[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2B[i]) for i in range(2*pad_len) ]
    v_sC = [ gf.mul(v_Cz[i],v_alpha[i]) ^ gf.mul(p_vec_z[i],v_p2C[i]) for i in range(2*pad_len) ]
    f_sA = gf.ifft( v_sA , 1 , 0 )
    f_sB = gf.ifft( v_sB , 1 , 0 )
    f_sC = gf.ifft( v_sC , 1 , 0 )
    s1 = gf.from_bytes( H.gen( h_state , bytes([1,2]) )[:gf.GF_BSIZE] )
    s2 = gf.from_bytes( H.gen( h_state , bytes([1,3]) )[:gf.GF_BSIZE] )
    s3 = gf.from_bytes( H.gen( h_state , bytes([1,4]) )[:gf.GF_BSIZE] )
    g_h = [ gf.mul(s1,f_sA[i])^gf.mul(s2,f_sB[i])^gf.mul(s3,f_sC[i]) for i in range(2*pad_len) ]
    ed = time.time()
    dump( "time:" , format(ed-st) , "secs" )
    dump( f"g_h: [{2*pad_len}] ...[{pad_len-4}:{pad_len+4}]", g_h[pad_len-4:pad_len+4] , "..." )

    ## row check
    dump( f"row check: calculate (f_Azxf_Bz - f_Cz) / Z_{pad_len}" )
    st = time.time()
    v_AzxBz = v_Cz[:pad_len] + [ cgf.gf264_mul(v_Az[i],v_Bz[i]) for i in range(pad_len,2*pad_len) ]
    f_Azxf_Bz = cgf.ibtfy( v_AzxBz , 0 )
    ed = time.time()
    dump( "time:", format(ed-st) , "secs" )
    dump( f"f_Azxf_Bz: [{len(f_Azxf_Bz)}] ...[{pad_len-4}:]" , f_Azxf_Bz[pad_len-4:pad_len+4] , "..." )
    dump( f"f_Cz     : [{len(f_Cz)}] ...[{pad_len-16}:]" , f_Cz[pad_len-4:] )
    dump( f"XXX: need a /Z_{pad_len} op. here" )

    ## generate f0 for fri_ldt and perform fri_ldt

    return False



##### test ##########

generate_proof( (mat_a,mat_b,mat_c,z,64) , b'123' )


m(#rows): 3656 x n: 3320, witness_idx: 64, pad_len: 4096
padding and calcuate f_1v, f_z, f_Az, f_Bz, f_Cz
time: 0.020545482635498047 secs
f_1v: [64] [1, 104, 6773987339827998880, 76, 6494990003695825333, 2770470512431615406, 9353829036159859324, 129, 3622001699451325254, 13826683270762018224, 6945490328438131111, 13170832442580444387, 13508198004833286135, 2430989766365671689, 4171295113934959528, 146, 1483026152508229671, 5060421071648320533, 14247327339982606064, 8149202561864665329, 2575143477663925702, 14879998630440192931, 7566345596596803686, 11768029609651875199, 2548218579848038601, 7145244881410666016, 10756402279887175656, 5427858226323345980, 2139843264328246966, 12479693107550326575, 14063864011514639672, 55, 12406569417845239693, 15795808078528240728, 14321077589967467932, 5798234984320495289, 9361449024593994014, 7796851569384837701, 11334226328499383373, 4860759622312203753, 2431339947958500416, 5784733405265691440, 2712770127487974368, 6262856457026633520, 1367427119097

False